# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [15]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [16]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [17]:
import os
from glob import glob

# Write your code below.
import sys
sys.path.append(os.getenv('SRC_DIR'))

# Load the PRICE_DATA environment variable
PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "**/part.0.parquet"), recursive=True)

# Load the data
dd_px = dd.read_parquet(parquet_files)

In [18]:
len(parquet_files)

11207

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Add lags for variables Close and Adj_Close
ddf = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag=x['Close'].shift(1),
        Adj_Close_lag=x['Adj Close'].shift(1)
    )
 )

# Add returns based on Adjusted Close
ddf['returns'] = (ddf['Adj Close'] / ddf['Adj_Close_lag']) - 1

# Add the following range
ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

# Assign the result to `dd_feat`
dd_feat = ddf


C:\Users\marky\AppData\Local\Temp\ipykernel_16992\492660799.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf = dd_px.groupby('ticker', group_keys=False).apply(


In [20]:
dd_feat

,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
npartitions=11207,,,,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [21]:
dd_feat.index


Dask Index Structure:
npartitions=11207
    object
       ...
     ...  
       ...
       ...
Name: ticker, dtype: object
Dask Name: assign-index, 19 graph layers

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [22]:
# Convert the Dask DataFrame to a Pandas DataFrame
df_pandas = dd_feat.compute()

# Make sure ticker is a column, not an index
if 'ticker' not in df_pandas.columns:
    df_pandas = df_pandas.reset_index()  # This will bring all index levels into columns

# Calculate 10-day rolling average return for each ticker
df_pandas['rolling_avg_return'] = (df_pandas.groupby('ticker')['returns']
                                           .rolling(10)
                                           .mean()
                                           .reset_index(level=0, drop=True))

# Display first few rows to verify
print(df_pandas.head())

Price ticker                      Date   Adj Close       Close        High  \
0        HUM 2023-01-03 00:00:00+00:00  492.875916  500.489990  511.010010   
1        HUM 2023-01-04 00:00:00+00:00  484.633301  492.119995  501.519989   
2        HUM 2023-01-05 00:00:00+00:00  480.595673  488.019989  493.369995   
3        HUM 2023-01-06 00:00:00+00:00  485.046906  492.540009  494.660004   
4        HUM 2023-01-09 00:00:00+00:00  474.381653  481.709991  499.839996   

Price         Low        Open   Volume       sector            subsector  \
0      489.670013  508.239990  1089500  Health Care  Managed Health Care   
1      484.750000  500.130005  1137100  Health Care  Managed Health Care   
2      483.709991  492.200012  1273800  Health Care  Managed Health Care   
3      488.029999  490.230011   977600  Health Care  Managed Health Care   
4      479.809998  495.570007  1065400  Health Care  Managed Health Care   

Price  year   Close_lag  Adj_Close_lag   returns  hi_lo_range  \
0      20



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It wasn’t necessary to switch to Pandas to calculate the moving average return because Dask can do this on its own. Dask is actually a better choice for large datasets because it’s built to handle big data efficiently by processing chunks of data separately, which avoids memory issues. Using Dask for these calculations keeps things simple, prevents crashes, and skips the extra step of converting to Pandas, which can slow things down.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.